In [246]:
# !pip install -U Selenium

In [247]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [248]:
url = 'https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%5E550&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&keywords='

In [249]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)

In [250]:
xpaths = {
    'cookies_reject_btn': "//button[@id='onetrust-reject-all-handler']",
    'rent_listing': "//div[@class='l-searchResult is-list']",
    'address': ".//address",
    'price': ".//*[@class='propertyCard-priceValue']",
    'beds': ".//*[@class='property-information']/span[contains(@class, 'bed')]/following-sibling::span[1]",
    'bath': ".//*[@class='property-information']/span[contains(@class, 'bath')]/following-sibling::span[1]",
    'link': ".//div[@class='propertyCard-details']/a",
    'next_page': "//button[@title='Next page']"
}

In [251]:
driver.find_element(By.XPATH, xpaths['cookies_reject_btn']).click()

In [252]:
def fetch_listings():
    data = []
    item_list = driver.find_elements(By.XPATH, xpaths['rent_listing'])
    for ele in item_list:
        link = ele.find_element(By.XPATH, xpaths['link']).get_attribute('href')
        address = ele.find_element(By.XPATH, xpaths['address']).text
        price = ele.find_element(By.XPATH, xpaths['price']).text
        try:
            beds = ele.find_element(By.XPATH, xpaths['beds']).text
        except Exception:
            beds = None
        try:
            bath = ele.find_element(By.XPATH, xpaths['bath']).text
        except Exception:
            bath = None
    
        data.append({
            'link': link,
            'address': address,
            'price': price,
            'beds': beds,
            'bath': bath
        })
    return data

In [253]:
final_structured_listings = []
for i in range(3):
    structured_listings = fetch_listings()
    print("iter:",i,len(structured_listings))
    element = driver.find_element(By.XPATH, "//a[@title='Contact us']")
    webdriver.ActionChains(driver).move_to_element(element).perform()
    driver.find_element(By.XPATH, xpaths['next_page']).click()
    final_structured_listings.extend(structured_listings)
    time.sleep(2)

iter: 0 25
iter: 1 25
iter: 2 25


In [254]:
driver.close()

In [255]:
len(final_structured_listings)

75

In [257]:
df = pd.DataFrame(final_structured_listings)

In [261]:
len(df['link'].unique())

75